In [ ]:
import xgboost as xgb

print("Dask Installed ?", xgb.dask.DASK_INSTALLED)

#client = xgb.dask.Client(n_workers=4, threads_per_worker=4)

In [ ]:
from dask.distributed import Client
client = Client()  # connect to cluster

In [ ]:
client = Client(n_workers=1, threads_per_worker=4)

In [ ]:
import dask_xgboost as dxgb

In [ ]:
print("Dask Installed ?", dxgb.DASK_INSTALLED)

In [ ]:
import xgboost as xgb
xgb.__version__

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import xgboost as xgb
import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
boston = load_boston()

In [3]:
boston_df = pd.DataFrame(data=boston.data, columns = boston.feature_names)
# Add one column with the target
boston_df["Price"] = boston.target
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
# Splitting the dataset
X_train, X_test, Y_train, Y_test = train_test_split(boston.data, boston.target, train_size=0.90, random_state=42)

In [5]:
dmat_train = xgb.DMatrix(X_train, Y_train, feature_names=boston.feature_names)
dmat_test = xgb.DMatrix(X_test, Y_test, feature_names=boston.feature_names)

In [6]:
tweedie_booster = xgb.train({'max_depth': 3, 'eta': 1, 'objective': 'reg:tweedie',
                             'tree_method':'hist', 'nthread':4,
                             'monotone_constraints':(1,0,1,-1,1,0,1,0,-1,1,1, -1, 1)},
                    dmat_train,
                    evals=[(dmat_train, "train"), (dmat_test, "test")])

[0]	train-tweedie-nloglik@1.5:28.3392	test-tweedie-nloglik@1.5:26.5961
[1]	train-tweedie-nloglik@1.5:19.3524	test-tweedie-nloglik@1.5:18.6182
[2]	train-tweedie-nloglik@1.5:18.7943	test-tweedie-nloglik@1.5:18.1855
[3]	train-tweedie-nloglik@1.5:18.7576	test-tweedie-nloglik@1.5:18.1568
[4]	train-tweedie-nloglik@1.5:18.7501	test-tweedie-nloglik@1.5:18.1531
[5]	train-tweedie-nloglik@1.5:18.7447	test-tweedie-nloglik@1.5:18.1528
[6]	train-tweedie-nloglik@1.5:18.7408	test-tweedie-nloglik@1.5:18.1524
[7]	train-tweedie-nloglik@1.5:18.7349	test-tweedie-nloglik@1.5:18.1893
[8]	train-tweedie-nloglik@1.5:18.7316	test-tweedie-nloglik@1.5:18.1863
[9]	train-tweedie-nloglik@1.5:18.7262	test-tweedie-nloglik@1.5:18.1955


In [7]:
print("\nTest  R2 Score : %.2f"%r2_score(Y_test, tweedie_booster.predict(dmat_test)))
print("Train R2 Score : %.2f"%r2_score(Y_train, tweedie_booster.predict(dmat_train)))


Test  R2 Score : 0.75
Train R2 Score : 0.89


In [8]:
from dask.distributed import Client
#When we create an instance of dask client without giving the IP address and port of scheduler, it'll create a cluster on the local machine itself.
client = Client(n_workers=1, threads_per_worker=4)

In [9]:
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:63704,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:63707,Total threads: 4
Dashboard: http://127.0.0.1:63708/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:63706,


In [13]:
params = {'max_depth': 3, 'eta': 1, 'objective': 'reg:tweedie',
                             'tree_method':'hist', 'nthread':4,
                             'monotone_constraints':(1,0,1,-1,1,0,1,0,-1,1,1, -1, 1)}
import dask_xgboost as dxgb
bst = dxgb.train(client, params, dmat_train, Y_train)

AttributeError: 'DMatrix' object has no attribute 'to_delayed'